In [1]:
Language = "English"
# Language = "Chinese"
mode = "train"
# mode = "test"
param_file = f"{Language}0"

# 如果要进行test，先把test.txt放到对应文件夹下，然后把mode改为"test"再运行

In [2]:
from dataprocess import build_vocab, build_tag2idx, data_process, set_log, combine_data
from HMM import HMM_model


set_log(None)
tag2idx = build_tag2idx(f"../NER/{Language}/tag.txt")
train_data, valid_data, test_data = data_process(f"../NER/{Language}", mode=mode)

train_path = f"../NER/{Language}/train.txt"
valid_path = f"../NER/{Language}/validation.txt"
test_path = f"../NER/{Language}/test.txt"
if mode == "train":
    vocab = build_vocab([train_path, valid_path])
elif mode == "test":
    vocab = build_vocab([train_path, valid_path, test_path])

model = HMM_model(tag2idx, vocab)
if mode == "train":
    model.train(train_data)
    model.save_param(param_file, format="npz")
elif mode == "test":
    model.load_param(param_file, format="npz")

if mode == "train":
    y_pred = model.valid(valid_data)
    valid_sentences = [sentence for sentence, _ in valid_data]
    combined_data = combine_data(valid_sentences, y_pred)
elif mode == "test":
    y_pred = model.valid(test_data)
    test_sentences = [sentence for sentence, _ in test_data]
    combined_data = combine_data(test_sentences, y_pred)

output_file = f"output_{Language}.txt"
with open(output_file, "w", encoding="utf-8") as file:
    file.write(combined_data)

2023-11-25 20:12:17,788 P12032 INFO train dataset size: 14041
2023-11-25 20:12:17,789 P12032 INFO valid dataset size: 3250


100%|██████████| 14041/14041 [00:00<00:00, 119866.53it/s]


In [3]:
from pathlib import Path
import sys

sys.path.append(str(Path.cwd().parent))

from NER.check import check

if mode == "train":
    check(
        language=Language,
        gold_path=f"../NER/{Language}/validation.txt",
        my_path=output_file,
    )
elif mode == "test":
    check(
        language=Language,
        gold_path=f"../NER/{Language}/test.txt",
        my_path=output_file,
    )

              precision    recall  f1-score   support

       B-PER     0.9651    0.6911    0.8054      1842
       I-PER     0.9532    0.7628    0.8474      1307
       B-ORG     0.8088    0.7442    0.7751      1341
       I-ORG     0.8589    0.6405    0.7338       751
       B-LOC     0.9126    0.8302    0.8694      1837
       I-LOC     0.8682    0.7432    0.8008       257
      B-MISC     0.9137    0.8037    0.8552       922
      I-MISC     0.8488    0.6329    0.7252       346

   micro avg     0.9025    0.7468    0.8173      8603
   macro avg     0.8912    0.7311    0.8016      8603
weighted avg     0.9054    0.7468    0.8165      8603

